In [ ]:
import train as t
import tensorflow as tf

import importlib
#importlib.reload(t)

from enum import Enum
from functools import reduce
import evaluation as e
import numpy as np

In [ ]:
import distance

#Exact_match and token_accuracy is adapted from the evaluation file
# in the tutorial on Neural Machine Translation: https://github.com/tensorflow/nmt

def exact_match(labels, predictions, max_token_check=None):
    #"""Compute exact match"""
    match = 0.0
    count = 0.0
    if max_token_check == None:
        max_token_check = 1000
    for idx in range(len(labels)):
        if np.all(labels[idx][:max_token_check] == predictions[idx][:max_token_check]):
            match += 1

        count += 1
    return 100 * match / count


def token_accuracy(labels, predictions, max_token_check=None):
    #"""Compute accuracy on per word basis."""
    total_acc, total_count = 0., 0.
    if max_token_check==None:
        m_length = 1000
    else:
        m_length = max_token_check

    for idx, target_sentence in enumerate(labels):
        prediction = predictions[idx]

        match = 0.0

        total_count += 1 
        for pos in range(min(len(target_sentence), len(prediction), m_length)):
            label = target_sentence[pos]
            pred = prediction[pos]
            if label == pred:
                match += 1
        
             
        if max_token_check==None:
            total_acc += 100 * match / max(len(target_sentence), len(prediction))
        else:
            total_acc += 100 * match / min(len(target_sentence), len(prediction), m_length)
            
    return total_acc / total_count


def lev_dist(labels, predictions, max_token_check=None):
    
    avg_distance = 0
    count = 0.0
  
    for idx in range(len(labels)):

        if max_token_check is not None:
            lev_distance = distance.levenshtein(labels[idx][:max_token_check], predictions[idx][:max_token_check])
            lev_distance = lev_distance / (min(len(labels[idx]), max_token_check))
        else:
            lev_distance = distance.levenshtein(labels[idx], predictions[idx])
            lev_distance = lev_distance / (len(labels[idx]))

        avg_distance = avg_distance + lev_distance
        count += 1

    avg_distance = float(avg_distance) / count
    return avg_distance


def get_metrics(labels, predictions, max_token_check=None):
    exact_match_avg = exact_match(labels, predictions, max_token_check)
    token_accuracy_avg = token_accuracy(labels, predictions, max_token_check)
    edit_distance_avg = lev_dist(labels, predictions, max_token_check)
    return exact_match_avg, token_accuracy_avg, edit_distance_avg


def get_token_seq(int_sequence):
    
    output = []
    for value in int_sequence:
        output.append(reverse_target_token_index[value])
        if reverse_target_token_index[value] == "**end**":
            break
    return output


In [ ]:
# Create the vocabulary
token_vocabulary = ["**end**", "**start**", "**unknown**"]

token_vocabulary.extend(t.get_vocabulary("train"))

target_tokens = token_vocabulary  # TODO: Refactor this. Currently duplicate naming

token_vocab_size = len(target_tokens)
# todo: document what was lifted from tutorials and what we wrote ourselves
target_token_index = dict(
    [(token, i) for i, token in enumerate(target_tokens)])

reverse_target_token_index = dict(
    (i, char) for char, i in target_token_index.items())



print("\n ======================= Loading Data =======================")
#new cell

train_dataset = t.get_data_somehow('train', True, t.hparams['mini_batch_size'], t.hparams['max_token_length'], t.hparams['max_train_num_samples'] , target_token_index)
val_dataset = t.get_data_somehow('val', True, t.hparams['mini_batch_size'], t.hparams['max_token_length'], t.hparams['max_val_num_samples'], target_token_index)

train_encoder_input_data_batches = train_dataset[0]
train_target_texts_batches = train_dataset[1]
train_sequence_lengths_batches = train_dataset[2]
train_decoder_input_data_batches = train_dataset[3]
train_decoder_target_data_batches = train_dataset[4]

val_encoder_input_data_batches = val_dataset[0]
val_target_texts_batches = val_dataset[1]
val_sequence_lengths_batches = val_dataset[2]
val_decoder_input_data_batches = val_dataset[3]
val_decoder_target_data_batches = val_dataset[4]
print("\n ======================= Data Loaded =======================")

In [ ]:
tf.reset_default_graph()


embedding_decoder, decoder_cell, decoder_initial_state, projection_layer, img \
= t.create_graph(token_vocab_size, t.hparams['num_units'], t.hparams['use_attention'], t.hparams['use_encoding_average_as_initial_state'], False)


In [ ]:
sess = tf.Session()

checkpoint = "model_8.ckpt"

ON_FLOYDHUB = True

if ON_FLOYDHUB:
    t.initialize_variables(sess, restore=True, path='/checkpoints/checkpoints/model_8.ckpt')
else:
    t.initialize_variables(sess, restore=True, path='../output/checkpoints/' + checkpoint)


In [ ]:
def get_predictions_and_labels(evaluation_images, evaluation_ground_truth, beam_size):    
    
    predictions = []
    labels = []    

    print("Evaluating " + str(len(evaluation_images)) + " batches in the " + EVALUATION_SET + " set")
    for idx, batch in enumerate(evaluation_images):


        if idx == 400:
            break

        if beam_size == 1:
            output, logits = t.predict_batch(sess,
                          batch,
                          target_token_index,
                          embedding_decoder,
                          decoder_cell,
                          decoder_initial_state,
                          projection_layer,
                          img)
        else:
            output, logits = t.predict_batch_beam(sess,
                          batch,
                          target_token_index,
                          embedding_decoder,
                          decoder_cell,
                          encoder_state,
                          projection_layer,
                          img,
                          beam_size=beam_size)

        if idx % 20 == 0:
            print("Progress: " + str(idx) + " batches")

        for idy in range(output.shape[0]):
            prediction = get_token_seq(output[idy,:])
            predictions.append(prediction[:-1])
            labels.append(evaluation_ground_truth[idx][idy].split()[1:])
        
    return predictions, labels

In [ ]:
EVALUATION_SET = "VAL"
if EVALUATION_SET == "VAL":
    evaluation_images = val_encoder_input_data_batches
    evaluation_ground_truth = val_target_texts_batches
elif EVALUATION_SET == "TRAIN":
    evaluation_images = train_encoder_input_data_batches
    evaluation_ground_truth = train_target_texts_batches

In [ ]:
predictions1, labels = get_predictions_and_labels(evaluation_images, evaluation_ground_truth, beam_size=1)
metrics1 = get_metrics(labels, predictions)
print(metrics1)

In [ ]:
def get_validation_loss():
    #num_val_batches = len(val_sequence_lengths_batches)
    val_loss = 0    
    for i in range(num_val_batches):
        input_data = {img: val_encoder_input_data_batches[i],
                                        decoder_lengths: val_sequence_lengths_batches[i],
                                         decoder_inputs: val_decoder_input_data_batches[i],
                                          decoder_outputs: val_decoder_target_data_batches[i],
                                         }
   
        output_tensors = [train_loss]
        loss = sess.run(output_tensors, 
                               feed_dict=input_data)
        
        print(loss)
        val_loss = val_loss + loss[0]
        
    val_loss = val_loss / len(val_decoder_input_data_batches[i])
    return val_loss